# I. Initialize Notebook

This Notebook is used to run the WaPOR-based Rapid Water Accounting+ Sheet 1 analysis.


## 1. Requirements 


In [ ]:
!pip install WaporIHE
!pip install gdal
!pip install netcdf4
!pip install pyshp

## 2. Import packages and functions


**From installed packages**

In [ ]:
import os
import shutil
import glob
import xarray as xr
import numpy as np
import pandas as pd
import calendar
import datetime
import WaporIHE
import gdal
import ogr
import shapefile
import geopy
import tempfile
import csv
from matplotlib import pyplot as plt

**From WA_Sheet1 GitHub repo**

In [ ]:
%cd /content/
if os.path.exists('/content/WA_Sheet1'):
  shutil.rmtree('/content/WA_Sheet1') 
!git clone https://github.com/trngbich/WA_Sheet1.git
import WA_Sheet1
from WA_Sheet1.pickle_basin import *
from WA_Sheet1.LCC_to_LUWA import *
from WA_Sheet1 import GIS_functions as gis
from WA_Sheet1.create_NC import make_netcdf,get_lats_lons
from WA_Sheet1.model_SMBalance import run_SMBalance,open_nc,merge_yearly_nc
from WA_Sheet1.average_by_LU import Total_perLU
from WA_Sheet1 import grace_functions as gf

## 3. Load data from Drive

**First, mount to Google drive folder**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Second, copy the files from drive folder to work space**

**MUST** "Add to My Drive"
this Drive folder -> Link: https://drive.google.com/drive/folders/1OUKQOIHmtIfXIt5ACGzMCcf73xsGlXMw?usp=sharing


In [ ]:
path_to_drive_folder=r'/content/drive/My Drive/WA/WA_Sheet1/.' #Change the path to where you add the data folder
!cp -a '$path_to_drive_folder' '/content/'

#II. Initilize Basin 

## 1. Set up working folders for basin

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 23 12:36:04 2019

@author: ntr002
"""
WORKING_DIR=os.getcwd()
MAIN_DIR=os.path.join(WORKING_DIR,'Main')
INPUT_DIR=os.path.join(WORKING_DIR,'Input')

### Global Input Info
GRaND_SHP=os.path.join(INPUT_DIR,'Global','GRanD','GRanD_reservoirs_v1_1.shp')
WDPA_SHP=os.path.join(INPUT_DIR,'Global','WDPA','WDPA_CatIandII_17countries.shp')
GRACE_data=os.path.join(INPUT_DIR,'Global','GRACE','GSFC.glb.200301_201607_v02.4-ICE6G')
ThetaSat_TIF=os.path.join(INPUT_DIR,'Global','HiHydroSoils','thetasat_topsoil.tif')

### Basin Info
BASIN_NAME='Litani'
start_date='2009-01-01'
end_date='2018-12-31'
end_month='AUG'

Qout_csv=r'/content/Input/Litani/Qoutlet_litani_QASMIYE_SeaMouth.csv'
Qibt_csv=r'/content/Input/Litani/Qibt_litani_MarkabaTunnel.csv'
BASIN_SHP=os.path.join(INPUT_DIR,'Global','Basins','{0}.shp'.format(BASIN_NAME))
unreserv_SHP=None #to adjust Basin reservoir map
makereserv_SHP=None #to adjust Basin reservoir map
GRACEMonthly_CSV=os.path.join(INPUT_DIR,'Global','GRACE','{0}_GRACE.csv'.format(BASIN_NAME))

###Create Basin folders
INPUT_FOLDER=os.path.join(INPUT_DIR,BASIN_NAME)
MAIN_FOLDER=os.path.join(MAIN_DIR,BASIN_NAME)
for folder in [INPUT_FOLDER,MAIN_FOLDER]:
  if not os.path.exists(folder):
    os.makedirs(folder)


## 2. Download WaPOR data to Working folder


**Instead of downloading from WaPOR API, copy sample (only Litani) data from Drive**

MUST "Add to My Drive" this Drive folder -> Link: https://drive.google.com/drive/folders/1Z3FkWQlUzoBFD6dlbkWKIjJ-pwb1w3yy?usp=sharing

In [ ]:
path_to_drive_folder=r'/content/drive/My Drive/WA/WA_Sheet1_Input_Litani/.' #Change the path to where you add the data folder
!cp -a '$path_to_drive_folder' '$INPUT_FOLDER'

**OR Download new dataset from WaPOR**

In [ ]:
shape=shapefile.Reader(BASIN_SHP)
xmin,ymin,xmax,ymax=shape.bbox

Token=input('Your WaPOR API Token: ')

# WaporIHE.PCP_daily(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25],version=2,level=1)

# WaporIHE.PCP_monthly(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25],version=2,level=1)

# WaporIHE.RET_monthly(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5],version=2,level=1)

# WaporIHE.AET_monthly(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],version=2,level=2)

# WaporIHE.LCC_yearly(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],version=2,level=2)

# WaporIHE.I_dekadal(APIToken=Token,Dir=INPUT_FOLDER,Startdate='2009-01-01',
#                   Enddate='2010-01-01',latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],version=2,level=2)

## 3. Create metadata dictionary

In [ ]:
BASIN={'Name':BASIN_NAME,
       'time_range':[start_date,end_date],
       'end_month':end_month,
       'Dir':MAIN_FOLDER,       
        'geo_data':{
                   'basin':BASIN_SHP,
                   'unreserv':unreserv_SHP, 
                   'makereserv':makereserv_SHP,
                      },
        'global_data':{                     
                     'grand':GRaND_SHP,
                     'wdpa':WDPA_SHP,
                     'grace':GRACE_data
                     },
        'input_data':{
                      'yearly':{
                              'lcc':[r""+os.path.join(INPUT_FOLDER,'L2_LCC_A'),
                                     '-','Landcover Class'],
                              },
                      'monthly':{
                              'p':[r""+os.path.join(INPUT_FOLDER,'L1_PCP_M'),
                                   'mm/month','Precipitation'],
                              'et':[r""+os.path.join(INPUT_FOLDER,'L2_AETI_M'),
                                    'mm/month','Actual Evapotranspiration'],
                              'ret':[r""+os.path.join(INPUT_FOLDER,'L1_RET_M'),
                                    'mm/month','Reference Evapotranspiration'],   
                              'i': [r""+os.path.join(INPUT_FOLDER,'L2_I_M'),
                                    'mm/month','Interception'],                                                                   
                              },  
                    'stat':{                                                          
                            'area':None,                 
                            'thetasat':ThetaSat_TIF
                            }                                                   
                              },
        'input_ts':{
                'Qout':Qout_csv,
                'Qibt':Qibt_csv
                    },
        'main_data':{
                'yearly':{},
                     'monthly':{},
                     'stat':{},
                     }                
                }

pickle_out(BASIN)
BASIN

In [ ]:
BASIN

# III. Compute intermediate data

## 1. Calculate monthly Interception from L2_I_D
**If L2_I_M is not available via API**

In [ ]:
#Get list of dekadal rasters
Dekadal_I_folder=os.path.join(INPUT_FOLDER,'L2_I_D')
input_fhs=glob.glob(os.path.join(Dekadal_I_folder,'*.tif'))

#Get month dates
start_date=BASIN['time_range'][0]
end_date=BASIN['time_range'][1]
month_dates=pd.date_range(start_date,end_date,freq='M')

#Get df avail
ds_code='L2_I_D'
WaporIHE.download.API.setAPIToken(Token)
df_avail=WaporIHE.download.API.getAvailData(ds_code,time_range='{0},{1}'.format(start_date,end_date),version=2,level=2)

output_folder=os.path.join(INPUT_FOLDER,'L2_I_M')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(input_fhs[0])
for date in month_dates:
    month_fhs=[]
    for in_fh in input_fhs:
        raster_id=os.path.split(in_fh)[-1].split('.tif')[0][-9:]
        raster_info=df_avail.loc[df_avail['raster_id']==raster_id]
        timestr=raster_info['DEKAD'].iloc[0]
        year=int(timestr[0:4])
        month=int(timestr[5:7])
        if (year==date.year)&(month==date.month):
            month_fhs.append(in_fh)
    SumArray=np.zeros((ysize,xsize),dtype=np.float32)
    for fh in month_fhs:
        Array=gis.OpenAsArray(fh,nan_values=True)
        SumArray+=Array
    out_fh=os.path.join(output_folder,'L2_I_{:04d}{:02d}.tif'.format(date.year,date.month))    
    gis.CreateGeoTiff(out_fh, SumArray, driver, NDV, xsize, ysize, GeoT, Projection)

BASIN['input_data']['monthly']['i']=[r""+os.path.join(INPUT_FOLDER,'L2_I_M'),'mm/month','Interception']
pickle_out(BASIN)

##2. Calculate number of rainy days per month

In [ ]:
Data_Path_P=os.path.join(INPUT_FOLDER,'L1_PCP_E')

Startdate=BASIN['time_range'][0]
Enddate=BASIN['time_range'][1]

Data_Path_RD = os.path.join(BASIN['Dir'],'data', 'Rainy_Days')
if not os.path.exists(Data_Path_RD):
    os.makedirs(Data_Path_RD)

Dates = pd.date_range(Startdate, Enddate, freq ='MS')
os.chdir(Data_Path_P)

for Date in Dates:
    # Define the year and month and amount of days in month
    year = Date.year
    month = Date.month
    daysinmonth = calendar.monthrange(year, month)[1]

    # Set the third (time) dimension of array starting at 0
    i = 0

    # Find all files of that month
    files = glob.glob('*daily_%d.%02d.*.tif' %(year, month))

    # Check if the amount of files corresponds with the amount of days in month
    if len(files) is not daysinmonth:
        print('ERROR: Not all Rainfall days for month %d and year %d are downloaded'  %(month, year))

    # Loop over the days and store data in raster
    for File in files:
        dir_file = os.path.join(Data_Path_P, File)

        # Get array information and create empty numpy array for daily rainfall when looping the first file
        if File == files[0]:

            # Open geolocation info and define projection
            driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(dir_file)

            # Create empty array for the whole month
            P_Daily = np.zeros([daysinmonth,ysize, xsize],dtype=np.float32)

        # Open data and put the data in 3D array
        Data = gis.OpenAsArray(dir_file,nan_values=True)

        # Remove the weird numbers
        Data[Data<0] = 0

        # Add the precipitation to the monthly cube
        P_Daily[i, :, :] = Data
        i += 1

    # Define a rainy day
    P_Daily[P_Daily > 0.201] = 1
    P_Daily[P_Daily != 1] = 0

    # Sum the amount of rainy days
    RD_one_month = np.nansum(P_Daily,0)

    # Define output name
    Outname = os.path.join(Data_Path_RD, 'Rainy_Days_NumOfDays_monthly_%d.%02d.01.tif' %(year, month))

    # Save tiff file
    gis.CreateGeoTiff(Outname, RD_one_month, driver, NDV, xsize, ysize, GeoT, Projection)
    
BASIN['input_data']['monthly']['nRD']=[Data_Path_RD,'days/month','Number of rainy days']
pickle_out(BASIN)

##3. Reclassify WaPOR LCC to LUWA

In [ ]:
"""
Created on Thu Aug  8 13:42:44 2019

@author: ntr002
WAPOR LCC to LUWA

Note: start Jupyter notebook with "conda activate env"
"""
### Get all yearly WaPOR LCC maps
LCC_fhs=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))
WaPOR_LCC=LCC_fhs[0] #template

### Create Reservoir raster map for basin
Dir_out=os.path.join(BASIN['Dir'],'data','Reservoir')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
Basin_Reservoir_tif=os.path.join(Dir_out,'Reservoir_basin.tif')
Global_GRaND_Reservoir=BASIN['global_data']['grand']

# adjust reservoir
Resrv_to_Lake=BASIN['geo_data']['unreserv']
Lake_to_Reserv=BASIN['geo_data']['makereserv']  
if (Resrv_to_Lake!=None)&(Lake_to_Reserv!=None): #require 2 shapefiles to unreservoir and makereservoir           
    Adjust_GRaND_reservoir(Basin_Reservoir_tif,WaPOR_LCC,
                           Global_GRaND_Reservoir,Resrv_to_Lake,Lake_to_Reserv)    
else:                
    Rasterize_shapefile(Global_GRaND_Reservoir,WaPOR_LCC,Basin_Reservoir_tif)
    
BASIN['geo_data']['reservoir']=Basin_Reservoir_tif

### Create Protected Area raster map for basin
Dir_out=os.path.join(BASIN['Dir'],'data','Protected')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
ProtectedArea_tif=os.path.join(Dir_out,'ProtectedArea_basin.tif')
WDPA=BASIN['global_data']['wdpa']
Rasterize_shapefile(WDPA,WaPOR_LCC,ProtectedArea_tif)

### Reclassify LCC to LUWA       
Dir_out=os.path.join(BASIN['Dir'],'data','luwa')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for fh in LCC_fhs:
    Reclass_LCC_to_LUWA(fh,Dir_out,ProtectedArea_tif,Basin_Reservoir_tif)

# Add LUWA folder to basin metadata
BASIN['input_data']['yearly']['lu']=[Dir_out,'-','LUWA Categories']
pickle_out(BASIN)  

# IV. Create netCDF for each variable

In [ ]:
BASIN['input_data']

In [ ]:
#Get inputs for create_NC
cutline=BASIN['geo_data']['basin']
Dir_out=os.path.join(BASIN['Dir'],'data','nc')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)    
template=glob.glob(os.path.join(BASIN['input_data']['monthly']['p'][0],'*.tif'))[0]          
    
## yearly maps
# for key in ['p','et']:#BASIN['input_data']['yearly']:             
#     if BASIN['input_data']['yearly'][key] is not None:
#         nc_fn=os.path.join(Dir_out,key+'_yearly.nc')        
#         dataset={key:[BASIN['input_data']['yearly'][key][0],
#                        ('time','latitude', 'longitude'), 
#                        {'units': BASIN['input_data']['yearly'][key][1],                                 
#                         'quantity':BASIN['input_data']['yearly'][key][2],
#                         'source': 'WaPOR', 'period':'year'}]}
#         succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='year')
#         if succes:
#             BASIN['main_data']['yearly'][key]=nc_fn
#             print('Finished {0}_yearly.nc'.format(key))
            
### monthly maps  
for key in ['p','et']:#BASIN['input_data']['monthly']:           
    if BASIN['input_data']['monthly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_monthly.nc')                
        dataset={key:[BASIN['input_data']['monthly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['monthly'][key][1],                                 
                        'quantity':BASIN['input_data']['monthly'][key][2],
                        'source': 'WaPOR', 'period':'month'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='month')
        if succes:
            BASIN['main_data']['monthly'][key]=nc_fn
            print('Finished {0}_monthly.nc'.format(key))
  
## static maps
lats, lons, optionsProj, optionsClip =get_lats_lons(template,cutline)
Dir_out=os.path.join(BASIN['Dir'],'data','stat')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)
for key in BASIN['input_data']['stat']:            
    if BASIN['input_data']['stat'][key] is not None:  
        basename= os.path.basename(BASIN['input_data']['stat'][key])
        outfn=os.path.join(Dir_out,basename)  
        temp_file = tempfile.NamedTemporaryFile(suffix='.tif').name               
        gdal.Warp(temp_file, BASIN['input_data']['stat'][key], options = optionsProj)
        gdal.Warp(outfn, temp_file, options = optionsClip)
        os.remove(temp_file)
        BASIN['main_data']['stat'][key]=outfn 
        print('Finished {0}'.format(basename))

pickle_out(BASIN)   
    

## Correct ET of water bodies by ET reference
**If ETa of water bodies is underestimated**

In [ ]:
#%% replace Et of water bodies by ET reference (based on FAO's recommendation)
#lu
lcc,_=open_nc(BASIN['main_data']['yearly']['lcc'])
#ET
et,_ = open_nc(BASIN['main_data']['monthly']['et'])
#RET
ret,_= open_nc(BASIN['main_data']['monthly']['ret'])

lcc_dict = {
                20: 'Shrubland',
                30: 'Grassland', 
                41: 'Cropland, rainfed',
                42: 'Cropland, irrigated or under water management', 
                43: 'Cropland, fallow', 
                50: 'Built-up', 
                60: 'Bare / sparse vegetation', 
                70: 'Permament snow / ice', 
                80: 'Water bodies', 
                81: 'Temporary water bodies', 
                90: 'Shrub or herbaceous cover, flooded', 
                111: 'Tree cover: closed, evergreen needle-leaved', 
                112: 'Tree cover: closed, evergreen broadleaved', 
                114: 'Tree cover: closed, deciduous broadleaved', 
                115: 'Tree cover: closed, mixed type', 
                116: 'Tree cover: closed, unknown type', 
                121: 'Tree cover: open, evergreen needle-leaved', 
                122: 'Tree cover: open, evergreen broadleaved', 
                123: 'Tree cover: open, deciduous needle-leaved', 
                124: 'Tree cover: open, deciduous broadleaved', 
                125: 'Tree cover: open, mixed type',
                126: 'Tree cover: open, unknown type', 
                200: 'Sea water'
          }
##
et_lcc_t = []

for i in range(len(lcc.time)): 
    t1 = i*12
    t2 = (i+1)*12
    lcc_t = lcc.isel(time = i)
    et_m_t = []
    for t in range(t1,t2):
        et_t = et.isel(time = t)  
        ret_t = ret.isel(time = t)  
        et_updated_t=et_t.where((lcc_t!=80)&(lcc_t!=200),ret_t)
        #et_updated_t.assign_coords({'time': et_t.time})
        et_updated_t = et_updated_t.assign_coords(time=et_t.time)
        et_updated_t = et_updated_t.expand_dims('time')
        et_m_t.append(et_updated_t)
    et_m = xr.concat(et_m_t, dim = 'time')
    et_lcc_t.append(et_m)
et_modified= xr.concat(et_lcc_t, dim='time')  
et_modified.name='Actual Evapotranspiration'
et_modified=et_modified.chunk({"latitude":-1,"longitude":-1}).to_dataset()

### save corrected ETa as netCDF
nc_fn=r"et_corrected_monthly.nc"
outfolder = os.path.join(BASIN['Dir'],'data','nc')
nc_path=os.path.join(outfolder,nc_fn)
et_modified.to_netcdf(nc_path,encoding={"Actual Evapotranspiration":{'zlib':True}})
BASIN['main_data']['monthly']['et_corrected']=nc_path
pickle_out(BASIN)

#V. Pixel-analysis: Run Soil Moisture Balance model to compute ET$_{incr}$ and ET$_{rain}$

## 1. Run Model

In [ ]:
## input data
smsat_file = BASIN['main_data']['stat']['thetasat']
## read nc files    
p_in = BASIN['main_data']['monthly']['p']
e_in = BASIN['main_data']['monthly']['et_corrected'] #use corrected et 
#e_in = BASIN['main_data']['monthly']['et'] #use original et
i_in = BASIN['main_data']['monthly']['i']
rd_in = BASIN['main_data']['monthly']['nRD']
lu_in = BASIN['main_data']['yearly']['lcc']

## Run
output_folder=os.path.join(BASIN['Dir'],'data','nc')
(etrain_dir,etincr_dir)=run_SMBalance(output_folder,p_in,e_in,i_in,rd_in,lu_in,smsat_file,start_year=2009)

##2. Merge resulted monthly ET$_{incr}$ and ET$_{rain}$

In [ ]:
BASIN['main_data']['monthly']['etrain']=os.path.join(output_folder,'etrain_monthly.nc')
BASIN['main_data']['monthly']['etincr']=os.path.join(output_folder,'etincr_monthly.nc')
merge_yearly_nc(etrain_dir,BASIN['main_data']['monthly']['etrain'],varname='ET Rainfall')
merge_yearly_nc(etincr_dir,BASIN['main_data']['monthly']['etincr'],varname='ET Incremental')
pickle_out(BASIN)

# VI. Aggregate annual value by hydrological year

In [ ]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['ret','et_corrected','i']:
    nc=BASIN['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(BASIN['Dir'],'data','nc')  
    attrs={"units":"mm/year", 
           "source": "Hydrolological year aggregation from {0}".format(nc), 
           "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    BASIN['main_data']['yearly'][key]=nc_path

In [ ]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['p','et','etrain','etincr']:
    nc=BASIN['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(BASIN['Dir'],'data','nc')  
    attrs={"units":"mm/year", 
           "source": "Hydrolological year aggregation from {0}".format(nc), 
           "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    BASIN['main_data']['yearly'][key]=nc_path

# V. Calculate volume fluxes table for Sheet 1

##1. Select Region of Interest

**Calculate area (km2) map**

In [ ]:
# get template from LU map
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],"*.tif"))[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(template)
# calculate area per pixel
area_map=gis.MapPixelAreakm(template)
# save area map as tif
BASIN['input_data']['stat']['area']=os.path.join(BASIN['Dir'],'data','stat','Area_km.tif')
gis.CreateGeoTiff(BASIN['input_data']['stat']['area'],area_map,driver, NDV, xsize, ysize, GeoT, Projection)

# create area mask
shape=BASIN['geo_data']['basin']
area=BASIN['input_data']['stat']['area']
BASIN['main_data']['stat']['area']=os.path.join(BASIN['Dir'],'data','stat','Basin_Area_km.tif')
gis.Clip_shapefile(area,shape,BASIN['main_data']['stat']['area'])

**Select Basin area**

In [ ]:
area_fh=BASIN['main_data']['stat']['area']
# area_fh=Basin['main_data']['subbasin']
area=gis.OpenAsArray(area_fh,nan_values=True)

## 2. Calculate P, ET, ETincr, ETrain and per LUWA category



**Calculate total P, ET, ETincr, ETrain**

In [ ]:
ts_all=[] #all timeseries
for key in ['p','et','etrain','etincr']:
    nc=BASIN['main_data']['yearly'][key]
    dts=xr.open_dataset(nc)
    var_name=list(dts.keys())[0]
    var=dts[var_name].chunk({"time": 1, "latitude": 1000, "longitude": 1000}).ffill("time")
    Volume=var*area
    ts=Volume.sum(dim=['latitude','longitude']).to_dataframe()
    ts.index=[y.year for y in ts.index]
    ts_all.append(ts)

**Calculate ETincr, ETrain per LU category**

In [ ]:
LU,_=open_nc(BASIN['main_data']['yearly']['lu'])
ETrain,_=open_nc(BASIN['main_data']['yearly']['etrain'])
ETincr,_=open_nc(BASIN['main_data']['yearly']['etincr'])

### Different year date to year 
LU=LU.groupby('time.year').first(skipna=False)
ETincr=area*ETincr.groupby('time.year').first(skipna=False)
ETrain=area*ETrain.groupby('time.year').first(skipna=False)

### average per LU
ts_ETincr=Total_perLU(ETincr,LU)
ts_ETrain=Total_perLU(ETrain,LU)

ts_all.append(ts_ETincr)
ts_all.append(ts_ETrain)

BASIN['main_data']['ts_all']=ts_all
pickle_out(BASIN)

## 3. Calculate dS from GRACE 

**Calculate monthly dS**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 10 16:51:44 2018

Need to download global GRACE GFSC product from
https://ssed.gsfc.nasa.gov/grace/products.html
GSFC.glb.200301_201607_v02.3b-ICE6G - ASCII

@author: Claire Michailovsky 
"""

BASIN_SHP=BASIN['geo_data']['basin']
OUT_CSV=os.path.join(BASIN['Dir'],'grace','GRACE_monthly.csv')    
os.makedirs(os.path.join(BASIN['Dir'],'grace'))
MASCON_DATA_FOLDER = BASIN['global_data']['grace']

BUFFER_SHP = OUT_CSV.split('.')[:-1][0]+'_buffer.shp'
MASCON_SHP = OUT_CSV.split('.')[:-1][0]+'_mascons.shp'

MASCON_INFO = os.path.join(MASCON_DATA_FOLDER, 'mascon.txt')
MASCON_SOLUTION = os.path.join(MASCON_DATA_FOLDER, 'solution.txt')
MASCON_DATES = os.path.join(MASCON_DATA_FOLDER, 'time.txt')

BUFFER_DIST = .71
gf.create_buffer(BASIN_SHP, BUFFER_SHP, BUFFER_DIST)
df_info = pd.read_csv(MASCON_INFO, sep=r"\s+", header=None, skiprows=14,engine='python')
mascon_coords = zip(df_info[1], df_info[0])

df_dates = pd.read_csv(MASCON_DATES, sep=r"\s+", header=None, skiprows=13,engine='python')
fract_dates = df_dates[2]
mascon_dates = [str(gf.convert_partial_year(fdate)) for fdate in fract_dates]
#?? Return null geometry sometimes??
index_mascons_of_interest = gf.points_in_polygon(BUFFER_SHP, mascon_coords)
#??
data_lines = []
with open(MASCON_SOLUTION) as fp:
    for i, line in enumerate(fp):
        if i in np.array(index_mascons_of_interest) + 7:
            data_lines.append(np.array(line.rstrip('\n').rstrip().split(' ')).astype(float))

# Create .shp of mascon areas
# Adapeted from bec's SortGRACE.py
w = shapefile.Writer(MASCON_SHP, shapeType=shapefile.POLYGON)
w.field('MASCON_ID', 'C', '40')

for mascon_index in index_mascons_of_interest[0]:
    ID = mascon_index+1
    lon_center = df_info[1][mascon_index]
    lat_center = df_info[0][mascon_index]
    lon_span = df_info[3][mascon_index]
    lat_span = df_info[2][mascon_index]
    w.poly([
            [[lon_center + .5 * lon_span, lat_center + .5 * lat_span],
             [lon_center - .5 * lon_span, lat_center + .5 * lat_span],
             [lon_center - .5 * lon_span, lat_center - .5 * lat_span],
             [lon_center + .5 * lon_span, lat_center - .5 * lat_span],
             [lon_center + .5 * lon_span, lat_center + .5 * lat_span]]
            ])
    w.record(ID,'Polygon')
w.close()
# Get weights from relative intersection area
basin_poly = ogr.Open(BASIN_SHP)
mascon_poly = ogr.Open(MASCON_SHP)

basin_lyr = basin_poly.GetLayer()
mascon_lyr = mascon_poly.GetLayer()
for b_feature in basin_lyr:
    ids = []
    int_area = []
    total_area = 0
    for m_feature in mascon_lyr:
        b_geom = b_feature.GetGeometryRef()
        m_geom = m_feature.GetGeometryRef()
        test = b_geom.Intersection(m_geom)
        ids.append(m_feature.GetField(0))
        int_area.append(test.GetArea())
        total_area += test.GetArea()
        
weights = np.array(int_area)/total_area

weighted_line = [data_lines[i] * weights[i] for i in range(len(data_lines))]
weighted_average = np.sum(weighted_line, 0)

with open(OUT_CSV, 'w',newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['date', 'Equivalent Water Height [mm]'])
    for date, value in zip(mascon_dates, weighted_average):
        spamwriter.writerow([date, value*10])

BASIN['input_ts']['grace_monthly']=OUT_CSV
pickle_out(BASIN)

**Calculate yearly dS**

$$\Delta T\overline WS _m ={TWSA _{m+1} - TWSA _{m-1}\over 2}$$ (Biancamaria et al., 2019)

In [ ]:
'''
Based on Biancamaria et al. (2019)

@author: ntr002
'''
###Read monthly csv
ts_grace=pd.read_csv(BASIN['input_ts']['grace_monthly'],sep=',',index_col=0)

###Interpolate daily data
ts_day=pd.DataFrame(index=pd.date_range(start='2003-01-06',end='2016-07-14',freq='D'))
ts_grace_daily=pd.merge(ts_day,ts_grace, left_index=True,right_index=True, how='outer')
ts_grace_interpolate_daily=ts_grace_daily.interpolate()
# ts_grace_interpolate_daily.plot(style='.-')

###Calculate cumulative sum of dS
ts_dS_daily=ts_grace_interpolate_daily.diff()
# ts_dS_daily.plot()
ts_cumsum_dS_daily=ts_dS_daily.cumsum()
# ts_cumsum_dS_daily.plot()

###Calculate monthly dS by second order central difference between cumsum dS
ts_cumsum_dS_monthly_firstday=ts_cumsum_dS_daily.resample('MS').first()
ts_cumsum_dS_monthly_firstday #take value of 1st day of every month
ts_dS_monthly=ts_cumsum_dS_monthly_firstday.diff(2).shift(-1)/2
# ts_dS_monthly

###Calculate (hydrological) yearly dS
ts_dS_yearly=ts_dS_monthly.resample('A-{0}'.format(end_month)).sum()
ts_dS_yearly.index=ts_dS_yearly.index.year

###Save results
out_csv=os.path.join(BASIN['Dir'],'grace','GRACE_yearly.csv')
ts_dS_yearly[6:-1].to_csv(out_csv,sep=';') #only from 2009 to 2015 since 2016 is not complete
BASIN['input_ts']['grace_yearly']=out_csv
pickle_out(BASIN)

**Convert dS to volume**

In [ ]:
df_dS_y=pd.read_csv(BASIN['input_ts']['grace_yearly'],sep=';',index_col=0)
Area_skm=np.nansum(area)
df_grace_ds=df_dS_y*Area_skm
#df_grace_ds.index=[y.year for y in df_grace_ds.index]
df_grace_ds=df_grace_ds.rename(columns={'Equivalent Water Height [mm]':'GRACE_dS'})
ts_all.append(df_grace_ds)

BASIN['main_data']['ts_all']=ts_all
pickle_out(BASIN)

## 4. Read Outflows Qout, Qibt (inter-basin transfer)


**Read Qout**

In [ ]:
if BASIN['input_ts']['Qout'] is None:
  df_Qout_y=ts_all[0]*0
  Qout=df_Qout_y.rename(colums={'Precipitation':'Qout'})
else:
  Qout_m=pd.read_csv(BASIN['input_ts']['Qout'],sep=';',index_col=0,skiprows=1)
  Qout_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y %H:%M') for y in Qout_m.index]
  Qout_m=Qout_m.replace(-9999,np.nan)
  Qout_y=Qout_m.resample('A-{0}'.format(end_month)).sum() #mean()
  Qout_y.index=[y.year for y in Qout_y.index]
  ##
  df_Qout_y=Qout_y.where(Qout_y.days>=365) ##remove years with missing data
  df_Qout_y=df_Qout_y['km3/month']*1000000 ##convert to TCM=km2*mm
  df_Qout_y=df_Qout_y.dropna()
  df_Qout_y.name='Qout'
  Qout=df_Qout_y.to_frame()

ts_all.append(Qout)
BASIN['main_data']['ts_all']=ts_all
pickle_out(BASIN)


**Read Qibt**

In [ ]:
if BASIN['input_ts']['Qibt'] is None:
  df_Qibt_y=ts_all[0]*0
  Qibt=df_Qibt_y.rename(colums={'Precipitation':'Qibt'})
else:
  Qibt_m=pd.read_csv(BASIN['input_ts']['Qibt'],sep=';',index_col=0,skiprows=1)
  Qibt_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y %H:%M') for y in Qibt_m.index]
  Qibt_m=Qibt_m.replace(-9999,np.nan)
  Qibt_y=Qibt_m.resample('A-{0}'.format(end_month)).sum() #mean()
  Qibt_y.index=[y.year for y in Qibt_y.index]
  ##
  df_Qibt_y=Qibt_y.where(Qibt_y.days>=365) ##remove years with missing data
  df_Qibt_y=df_Qibt_y['km3/month']*1000000 ##convert to TCM=km2*mm
  df_Qibt_y=df_Qibt_y.dropna()
  df_Qibt_y.name='Qibt'
  Qibt=df_Qibt_y.to_frame()

ts_all.append(Qibt)
BASIN['main_data']['ts_all']=ts_all
pickle_out(BASIN)


##5. Estimate error and export volume fluxes results

In [ ]:
for i in range(len(ts_all)):
    if i ==0:
        df=ts_all[i]        
    else:
        df=pd.merge(df,ts_all[i],left_index=True,right_index=True,how='outer')

df['WB_dS']=df['Precipitation']-df['Actual Evapotranspiration']-df['Qout']-df['Qibt']
df['dS_Error']=df['GRACE_dS']-df['WB_dS']
df.to_csv(os.path.join(BASIN['Dir'],'df_all.csv'),sep=';')
BASIN['main_data']['df_all']=os.path.join(BASIN['Dir'],'df_all.csv')
pickle_out(BASIN)
df

# VI. Generate Sheet 1

## 1. Create Yearly Sheet 1 csv

In [ ]:
#Get sheet1 csv template
csv_template=os.path.join(WORKING_DIR,'WA_Sheet1','csv','Sample_sheet1.csv')

# convert_unit=1000000 #from TCM to BCM
convert_unit=1000 #from TCM to MCM

df=pd.read_csv(BASIN['main_data']['df_all'],sep=';',index_col=0)
df

In [ ]:
df=df/convert_unit
csv_folder=os.path.join(BASIN['Dir'],'data','csv')
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

df_template=pd.read_csv(csv_template,sep=";")
csv_fhs=[]
for year in df.index[1:-1]:
    df_year=df_template.copy()
    df_year['VALUE']=0.0
    df_year.loc[0,'VALUE']=df.loc[year,'Precipitation']
#    if df.loc[year,'dS']>0:
#        df_year.loc[11,'VALUE']=df.loc[year,'dS']
#    else:
#        df_year.loc[10,'VALUE']=abs(df.loc[year,'dS'])
    df_year.loc[10,'VALUE']=-df.loc[year,'WB_dS']
    df_year.loc[12,'VALUE']=df.loc[year,'etrain-PLU']
    df_year.loc[13,'VALUE']=df.loc[year,'etrain-ULU']
    df_year.loc[14,'VALUE']=df.loc[year,'etrain-MLU']
    df_year.loc[15,'VALUE']=df.loc[year,'etrain-MWU']
    df_year.loc[16,'VALUE']=df.loc[year,'etincr-PLU']
    df_year.loc[17,'VALUE']=df.loc[year,'etincr-ULU']
    df_year.loc[18,'VALUE']=df.loc[year,'etincr-MLU']
    df_year.loc[19,'VALUE']=df.loc[year,'etincr-MWU']
    df_year.loc[20,'VALUE']=df.loc[year,'etincr-MWU']
    df_year.loc[21,'VALUE']=df.loc[year,'etincr']-df.loc[year,'etincr-MWU']
    df_year.loc[26,'VALUE']=df.loc[year,'Qibt']
    df_year.loc[22,'VALUE']=df.loc[year,'Qout']
    outcsv=os.path.join(csv_folder,'Sheet1_{0}.csv'.format(int(year)))
    df_year.to_csv(outcsv,sep=";",index=False)
    csv_fhs.append(outcsv)

## 2. Calculate mean annual Sheet 1

In [ ]:
#%% Average sheet 1
def average_sheet(sheet_folder,sheet_number,period):    
    """
    Calculate average of yearly WA+ sheets in csv format
    Applicable for Sheet2, 3a, 4, 5, 6
    
    Parameters
    -------------
    sheet_folder: str
        folder of all sheet csv
    sheet_number: int
        number from 1 to 6
    period: list
        period to calculate average [yyyy,yyyy]
    unit: str
        unit of values in average csv file (Mm3 or km3) default is Mm3
        
    Returns
    -------------
    outname: str
        path to average csv
    """
    
    sheet_indexcol = {1: (0,1,2),
                   2: (0,1),
                   3: (0,1,2,3,4),
                   4: (0),
                   5: (0,1,3),
                   6: (0,1)}
    year_ls = pd.date_range(start='1/1/{0}'.format(period[0]),end='12/31/{0}'.format(period[1]),freq='Y')
    
    df_avg=0  
    for year in year_ls:
        sheet_csv = glob.glob(os.path.join(sheet_folder,'*{0}.csv'.format(str(year.year))))[0]
        df=pd.read_csv(sheet_csv,sep=';',index_col=sheet_indexcol[sheet_number],na_values='nan')
        df_avg+=df
    df_avg=df_avg/len(year_ls)
    
    outname=os.path.join(sheet_folder,'sheet{0}_{1}-{2}.csv'.format(sheet_number,period[0],period[1]))
    df_avg.to_csv(outname,sep=';',na_rep='nan')
    return outname

csv=average_sheet(os.path.join(BASIN['Dir'],'data','csv'),1,[2010,2018])
